<a href="https://colab.research.google.com/github/ryanleeallred/DS-Unit-4-Sprint-3-Deep-Learning/blob/main/module1-rnn-and-lstm/LS_DS_431_RNN_and_LSTM_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

The French mathematician Emile Borel once mused that [**infinite monkeys typing for an infinite amount of time**](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type the complete works of William Shakespeare. Let's see if we can get there a bit faster, with the power of **Recurrent Neural Networks and LSTMs**!

Our goal in this projectis to build a **Shakespeare Sonnet Generator**.<br>
Given a prompt of a few words as input, its task is to generate follow-on text that reads like a Shakespeare Sonnet!<br>

To build our Sonnet Generator we will use a type of model called a **sequence model**. Given a short sequence, a sequence  model predicts the **most likely next item in the sequence**. Sequence models are astonishingly versatile and powerful, because the **sequence** we want to predict can be quite general! It can be composed of **words**, or of **characters**, or of **musical notes**, or of data points in a **time series** such as EKG voltages, or stock prices, or even a sequence of **DNA nucleotides**! 

We will train our model on the entire corpus of Shakespeare's Sonnets, and the model will learn from that data the most likely patterns of characters.

In [1]:
import random
import sys
import os

import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import LambdaCallback

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Bidirectional
from tensorflow.keras.layers import LSTM

%matplotlib inline

# import a custom text data preparation class
!wget https://raw.githubusercontent.com/LambdaSchool/DS-Unit-4-Sprint-3-Deep-Learning/main/module1-rnn-and-lstm/data_cleaning_toolkit_class.py
from data_cleaning_toolkit_class import data_cleaning_toolkit

zsh:1: command not found: wget


### Use `request` to pull data from a URL

[**Read through the request documentation**](https://requests.readthedocs.io/en/master/user/quickstart/#make-a-request) to learn how to download the Shakespeare Sonnets from the Gutenberg website. 


In [2]:
# download all of Shakespeare's Sonnets from the Project Gutenberg website

# here's the link for the sonnets
url_shakespeare_sonnets = "https://www.gutenberg.org/cache/epub/1041/pg1041.txt"

# use requests and the url to download all of the sonnets - save the result to `data`
data = requests.get(url_shakespeare_sonnets)


In [3]:
# extract the downloaded text from the requests object and save it to `raw_text_data`
# hint: take a look at the attributes of `data`
# YOUR CODE HERE
raw_text_data = data.text
#raise NotImplementedError()

In [4]:
# check the data type of `raw_text_data`
assert(type(raw_text_data)==str)

### Data Cleaning

In [5]:
# as usual, we need to clean up the messy data
raw_text_data[:3000]

"\ufeff\r\n    The Project Gutenberg eBook of Shakespeare's Sonnets\r\n    \r\nThis ebook is for the use of anyone anywhere in the United States and \r\nmost other parts of the world at no cost and with almost no restrictions \r\nwhatsoever. You may copy it, give it away or re-use it under the terms \r\nof the Project Gutenberg License included with this ebook or online \r\nat www.gutenberg.org. If you are not located in the United States, \r\nyou will have to check the laws of the country where you are located \r\nbefore using this eBook.\r\n\r\n\r\n\r\n    \r\n        Title: Shakespeare's Sonnets\r\n        \r\n        Author: William Shakespeare\r\n\r\n        \r\n        Release date: September 1, 1997 [eBook #1041]Most recently updated: February 27, 2023\r\n        Language: English\r\n        \r\n        \r\n    \r\n        \r\n            *** START OF THE PROJECT GUTENBERG EBOOK SHAKESPEARE'S SONNETS ***\r\n        \r\n\r\n\r\n\r\nTHE SONNETS\r\n\r\nby William Shakespeare\r\n\r\

In [6]:
# Which characters could we use with the split() method to split the text into lines?

# split the text into **lines** and save the result to `split_data`

# YOUR CODE HERE
split_data = raw_text_data.split('\r\n')

In [7]:
# we need to drop all the boiler plate text (i.e. titles and descriptions) as well as extra white spaces
# so that we are left with only the sonnets themselves 
# split_data[:37]
# split_data[-400:]

**Use list index slicing to remove the titles and descriptions, so we only have the sonnets.**


In [8]:
# we need to drop all the boilerplate text (i.e., titles and descriptions) as well as extra white spaces
# so that we are left with only the sonnets themselves 

# find index boundaries (start, end)so that 
# sonnets exist between these indices 
# titles and descriptions exist outside of these indices

# use index slicing to isolate the sonnet lines from the text - save the result to `sonnets`

# YOUR CODE HERE
sonnets = split_data[37:-400]

In [9]:
sonnets[-50:]

['CLII',
 '',
 'In loving thee thou know’st I am forsworn,',
 'But thou art twice forsworn, to me love swearing;',
 'In act thy bed-vow broke, and new faith torn,',
 'In vowing new hate after new love bearing:',
 'But why of two oaths’ breach do I accuse thee,',
 'When I break twenty? I am perjur’d most;',
 'For all my vows are oaths but to misuse thee,',
 'And all my honest faith in thee is lost:',
 'For I have sworn deep oaths of thy deep kindness,',
 'Oaths of thy love, thy truth, thy constancy;',
 'And, to enlighten thee, gave eyes to blindness,',
 'Or made them swear against the thing they see;',
 '    For I have sworn thee fair; more perjured I,',
 '    To swear against the truth so foul a lie.',
 '',
 'CLIII',
 '',
 'Cupid laid by his brand and fell asleep:',
 'A maid of Dian’s this advantage found,',
 'And his love-kindling fire did quickly steep',
 'In a cold valley-fountain of that ground;',
 'Which borrow’d from this holy fire of Love,',
 'A dateless lively heat, still to en

Notice that there are many lines that should not be counted as sonnets!

In [10]:
# these non-sonnet lines have far fewer characters than the actual sonnet lines?

sonnets[200:240]

['Since sweets and beauties do themselves forsake',
 'And die as fast as they see others grow;',
 '    And nothing ’gainst Time’s scythe can make defence',
 '    Save breed, to brave him when he takes thee hence.',
 '',
 'XIII',
 '',
 'O! that you were your self; but, love you are',
 'No longer yours, than you yourself here live:',
 'Against this coming end you should prepare,',
 'And your sweet semblance to some other give:',
 'So should that beauty which you hold in lease',
 'Find no determination; then you were',
 'Yourself again, after yourself’s decease,',
 'When your sweet issue your sweet form should bear.',
 'Who lets so fair a house fall to decay,',
 'Which husbandry in honour might uphold,',
 'Against the stormy gusts of winter’s day',
 'And barren rage of death’s eternal cold?',
 '    O! none but unthrifts. Dear my love, you know,',
 '    You had a father: let your son say so.',
 '',
 'XIV',
 '',
 'Not from the stars do I my judgement pluck;',
 'And yet methinks I have astro

In [11]:
# use your judgement to decide on a good value for  
#   the  minimum number of characters that a sonnet should have
#   call it n_chars
n_chars = 10

# Let's use that observation to filter out all the non-sonnet lines!
#    save results to `filtered_sonnets`
# Hint: use a list comprehension

# YOUR CODE HERE
filtered_sonnets = [line.strip() for line in sonnets if len(line) > n_chars]

In [12]:
# ok - much better!
# but we still need to remove all the punctuation and case normalize the text
filtered_sonnets[:50]

['From fairest creatures we desire increase,',
 'That thereby beauty’s rose might never die,',
 'But as the riper should by time decease,',
 'His tender heir might bear his memory:',
 'But thou, contracted to thine own bright eyes,',
 'Feed’st thy light’s flame with self-substantial fuel,',
 'Making a famine where abundance lies,',
 'Thyself thy foe, to thy sweet self too cruel:',
 'Thou that art now the world’s fresh ornament,',
 'And only herald to the gaudy spring,',
 'Within thine own bud buriest thy content,',
 'And tender churl mak’st waste in niggarding:',
 'Pity the world, or else this glutton be,',
 'To eat the world’s due, by the grave and thee.',
 'When forty winters shall besiege thy brow,',
 'And dig deep trenches in thy beauty’s field,',
 'Thy youth’s proud livery so gazed on now,',
 'Will be a tatter’d weed of small worth held:',
 'Then being asked, where all thy beauty lies,',
 'Where all the treasure of thy lusty days;',
 'To say, within thine own deep sunken eyes,',
 

### Use Custom Data Cleaning Tool 

Use one of the methods in the `data_cleaning_toolkit` to clean your data.

There is an example of this in the guided project.

In [13]:
# instantiate the data_cleaning_toolkit class - save result to `dctk`

# YOUR CODE HERE
dctk = data_cleaning_toolkit()

In [14]:
# use data_cleaning_toolkit to remove punctuation and to case normalize - save results to `clean_sonnets`

# YOUR CODE HERE
clean_sonnets = [dctk.clean_data(line) for line in filtered_sonnets]

In [15]:
# much better!
display(clean_sonnets[:50])
print(len(clean_sonnets))

['from fairest creatures we desire increase',
 'that thereby beautys rose might never die',
 'but as the riper should by time decease',
 'his tender heir might bear his memory',
 'but thou contracted to thine own bright eyes',
 'feedst thy lights flame with selfsubstantial fuel',
 'making a famine where abundance lies',
 'thyself thy foe to thy sweet self too cruel',
 'thou that art now the worlds fresh ornament',
 'and only herald to the gaudy spring',
 'within thine own bud buriest thy content',
 'and tender churl makst waste in niggarding',
 'pity the world or else this glutton be',
 'to eat the worlds due by the grave and thee',
 'when forty winters shall besiege thy brow',
 'and dig deep trenches in thy beautys field',
 'thy youths proud livery so gazed on now',
 'will be a tatterd weed of small worth held',
 'then being asked where all thy beauty lies',
 'where all the treasure of thy lusty days',
 'to say within thine own deep sunken eyes',
 'were an alleating shame and thriftle

2155


### Use Your Data Tool to Create Character Sequences 
for the LSTM model

We'll need the `create_char_sequences` method for this task. <br>
However, this method requires a parameter called `maxlen,` which is responsible for setting the maximum sequence length. 

So what would be a good sequence length, exactly? 

To answer that question, let's do some statistics! 

In [16]:
def calc_stats(corpus):
    """
    Calculates statistics on the length of every line in the sonnets
    """
    
    # write a list comprehension that calculates each sonnet's line length - save the results to `doc_lens` 

    # use numpy to calculate and return the mean, median, std, max, min of the doc lens - all in one line of code

    # YOUR CODE HERE
    doc_lens = [len(line) for line in corpus]
    return np.mean(doc_lens), np.median(doc_lens), np.std(doc_lens), np.max(doc_lens), np.min(doc_lens)

In [17]:
# sonnet line length statistics 
mean, med, std, max_, min_ = calc_stats(clean_sonnets)
mean, med, std, max_, min_ 

(40.88027842227378, 41.0, 4.043677533721869, 57, 27)

In [18]:
# from the results of the sonnet line length statistics, use your judgement to select a value for maxlen
#   hint -- a good value might be half the median length of a sonnet line
# use .create_char_sequences() to create sequences

# YOUR CODE HERE
maxlen = 20
dctk.create_char_sequences(clean_sonnets, maxlen)

Created 18047 sequences.


Take a look at the `data_cleaning_toolkit_class.py` file. 

In the first 4 lines of code in the `create_char_sequences` method, class attributes `n_features` and `unique_chars` are created. <br>
Let's call them in the cells below.

In [19]:
# number of input features for our LSTM model
print(dctk.n_features)

27


In [20]:
# unique characters that appear in our sonnets 
dctk.unique_chars

[' ',
 'e',
 't',
 'r',
 'd',
 'y',
 'o',
 'x',
 'k',
 'p',
 'v',
 'b',
 'm',
 'w',
 'n',
 'j',
 'f',
 'c',
 'a',
 'u',
 'i',
 'g',
 'z',
 'q',
 'h',
 'l',
 's']

## Time for Questions 

----
**Question 1:** 

Why are the `number of unique characters` (i.e., **dctk.unique_chars**) and the `number of model input features` (i.e., **dctk.n_features**) the same?

**Hint:** The model that we will shortly build here is very similar to the text generation model we built in the guided project.

**Answer 1:**

Our model is receiving input one character at a time, therefore we are essentially creating a classifier where the classes are english characters (letters and spaces)


**Question 2:**

Take a look at the printout of `dctk.unique_chars` one more time. Notice that there is a white space. 

Why is it desirable to have a white space as a possible character to predict?

**Answer 2:**

We want our model to output words, but we're technically outputing one character at a time, therefore we need to be able to output spaces

----

### Use Our Data Tool to Create X and Y Splits

You'll need the `create_X_and_Y` method for this task. 

In [21]:
# TODO: provide a walkthrough of data_cleaning_toolkit with unit tests that check for understanding 
X, y = dctk.create_X_and_Y()

![](https://miro.medium.com/max/891/0*jGB1CGQ9HdeUwlgB)

In [22]:
# notice that our input array isn't a matrix - it's a rank three tensor
X.shape

(18047, 20, 27)

In [24]:
y[0]

array([False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False])

In $X$.shape, we see three numbers (*n1*, *n2*, *n3*). What do these numbers mean?

Well, *n1* tells us the number of samples that we have. But what about the other two?

In [62]:
# first index returns a signle sample, which we can see is a sequence 
first_sample_index = 0 
X[first_sample_index]

array([[False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False,  True],
       [False, False, False, False, False, False,  True, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, False, False,  True, False, False, False,
        False, False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False,  True, False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, False,  True, False, False, False, False,
  

Notice that each sequence (i.e., $X[i]$ where $i$ is some index value) is `maxlen` long and <br>
has a number of features equal to `dctk.n_features`. <br>Let's try to understand this shape.

In [63]:
# each sequence is maxlen long and has dctk.n_features number of features
X[first_sample_index].shape

(20, 27)

**Each row corresponds to a character vector,** and there is `maxlen` number of character vectors. 

**Each column corresponds to a unique character,** and there are `dctk.n_features` number of features. 


In [64]:
# let's index for a single character vector 
first_char_vect_index = 0
X[first_sample_index][first_char_vect_index]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True])

Notice that there is a single `True` value, and all the rest of the values are `False`. 

This is a one-hot encoding for which character appears at each index within a sequence. Specifically, the cell above is looking at the first character in the sequence.

Only a single character can appear as the first character in a sequence, so there will be a single `True` value, and the rest will be `False`. 

Let's say that `True` appears in the $ith$ index; by  $ith$ index we mean some index in the general case. So how can we find out which character corresponds to?

To answer this question, we need to use the character-to-integer look-up dictionary. 

In [65]:
# take a look at the index to character dictionary
# if a TRUE appears in the 0th index of a character vector,
# then we know that whatever char you see below next to the 0th key 
# is the character that character vector is endcoding for
dctk.int_char

{0: 'y',
 1: 'd',
 2: 'z',
 3: 'j',
 4: 'k',
 5: 'l',
 6: 'r',
 7: 'a',
 8: 'n',
 9: 'e',
 10: 'w',
 11: 'b',
 12: 'c',
 13: ' ',
 14: 'o',
 15: 'x',
 16: 's',
 17: 't',
 18: 'v',
 19: 'i',
 20: 'u',
 21: 'h',
 22: 'q',
 23: 'm',
 24: 'g',
 25: 'p',
 26: 'f'}

In [66]:
# let's look at an example to tie it all together

seq_len_counter = 0

# index for a single sample 
for seq_of_char_vects in X[first_sample_index]:
    
    # get index with max value, which will be the one TRUE value 
    index_with_TRUE_val = np.argmax(seq_of_char_vects)
    
    print (dctk.int_char[index_with_TRUE_val])
    
    seq_len_counter+=1
    
print ("Sequence length: {}".format(seq_len_counter))

f
r
o
m
 
f
a
i
r
e
s
t
 
c
r
e
a
t
u
r
Sequence length: 20


## Time for Questions 

----
**Question 1:** 

In your own words, how would you describe the numbers from the shape printout of `X.shape` to a classmate?


**Answer 1:**

(a, b, c) corresponds to (number of char sequences, chars in a sequence, one-hot encoding of a char into our features)

----


### Build a Shakespeare Sonnet Text Generation Model

Now that we have prepped our data (and understood that process), let's finally build out our character generation model, similar to what we did in the guided project.<br>

First, we'll create a callback to monitor the training -- by printing a sample of text generated by the model at the end of each epoch.

Helper function to generate a sample character:

In [67]:
def sample(preds, temperature=1.0):
    """
    Helper function to generate a sample character
    Input is a predictions vector from our model, for example a set of 27 character probabilities
    Output is the index of the generated character 
    """
    # convert predictions to an array 
    preds = np.asarray(preds).astype('float64')

    # use the temperature hyper-parameter to "warp" (sharpen or spread out) the probability distribution 
    preds = np.log(preds) / temperature

    # use the softmax activation function to create a new list of probabilities 
    #   corresponding to the "warped" probability distribution
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)

    # Draw a single sample from a multinomial distribution, given these probabilities
    #   The sample will be a one-hot encoded character
    """ Notes on the np.random.multinomial() function 
       The first argument is the number of "trials" we want: 1 in this case
       The second argument is the list of probabilities for each character
       The third argument is number of sets of "trials" we want: again, 1 in this case
       By analogy with a dice-rolling experiment: 
          This "trial" consists of generating a single "throw" of a die with 27 faces;
             each face corresponds to a character and its associated probability
    """

    probas = np.random.multinomial(1, preds, 1)
    
    # return the index that corresponds to the max probability 
    return np.argmax(probas)


Create the `on_epoch_end` function to be passed into `LambdaCallback()`

In [68]:
def on_epoch_end(epoch, _):
    """"
    Function invoked at the end of each epoch. Prints the text generated by our model.
    """
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    

    # randomly pick a starting index 
    # will be used to take a random sequence of chars from `text`
    start_index = random.randint(0, len(text) - dctk.maxlen - 1)
    
    # this is our seed string (i.e. input seqeunece into the model)
    generated = ''

    # start the sentence at index `start_index` and include the next` dctk.maxlen` number of chars
    sentence = text[start_index: start_index + dctk.maxlen]

    # add to generated
    generated += sentence

    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    # use model to predict what the next maxlen chars should be that follow the seed string
    for i in range(maxlen):

        # shape of a single sample in a rank 3 tensor 
        x_dims = (1, dctk.maxlen, dctk.n_features)
        # create an array of zeros with shape x_dims
        # recall that python considers zeros and boolean FALSE as the same
        x_pred = np.zeros(x_dims)

        # create a seq vector for our randomly select sequence 
        # i.e. create a numerical encoding for each char in the sequence 
        for t, char in enumerate(sentence):
            # for sample 0 in seq index t and character `char` encode a 1 (which is the same as a TRUE)
            x_pred[0, t, dctk.char_int[char]] = 1

        # next, take the seq vector and pass into model to get a prediction of what the next char should be 
        preds = model.predict(x_pred, verbose=0)[0]
        # use the sample helper function to get index for next char 
        next_index = sample(preds)
        # use look up dict to get next char 
        next_char = dctk.int_char[next_index]

        # append next char to sequence 
        sentence = sentence[1:] + next_char 
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

In [69]:
# need this for on_epoch_end()
text = " ".join(clean_sonnets)
print(f'All of Shakespeare\'s sonnets comprise about {len(text)} characters')

All of Shakespeare's sonnets comprise about 90251 characters


Create the callback object

In [70]:
# create callback object that will print out text generation at the end of each epoch 
# use for real-time monitoring of model performance
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

----
### Build and Train Model

Build a text generation model using LSTMs. Feel free to reference the model used in the guided project. 

It is recommended that you train this model to at least 50 epochs (but more if you're computer can handle it). 

You are free to change up the architecture as you wish. 

Just in case you have difficultly training a model, there is a pre-trained model saved to a file called `trained_text_gen_model.h5` that you can load (in the same way that you learned how to load in Keras models in Sprint 2 Module 4). 

In [72]:
# build text generation model layer by layer 
# fit model

# YOUR CODE HERE
model = Sequential([
    LSTM(264, input_shape=(dctk.maxlen, dctk.n_features), return_sequences=True, activation='relu'),
    LSTM(128, activation='relu'),
    Dense(dctk.n_features, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam')

model.fit(X, y, batch_size=256, epochs=50, callbacks=[print_callback], workers=6)

Epoch 1/50
71/71 [==============================] - ETA: 0s - loss: 2.9106
----- Generating text after Epoch: 0
----- Generating with seed: "en pilgrimage but wh"
en pilgrimage but wh hc othnefwc   batr 
71/71 [==============================] - 9s 122ms/step - loss: 2.9106
Epoch 2/50
71/71 [==============================] - ETA: 0s - loss: 2.8308
----- Generating text after Epoch: 1
----- Generating with seed: "not marble nor the g"
not marble nor the g yo s ebrrhshkfht he
71/71 [==============================] - 9s 128ms/step - loss: 2.8308
Epoch 3/50
71/71 [==============================] - ETA: 0s - loss: 2.8034
----- Generating text after Epoch: 2
----- Generating with seed: "a makeless wife the "
a makeless wife the a al cdote dtbhttreb
71/71 [==============================] - 9s 131ms/step - loss: 2.8034
Epoch 4/50
71/71 [==============================] - ETA: 0s - loss: 2.6354
----- Generating text after Epoch: 3
----- Generating with seed: "mers time the teemin"
mers time the t

### Save the trained model to a file

In [73]:
# save trained model to file 
model.save("trained_text_gen_model.h5")

### Let's Play With Our Trained Model 

Now that we have a trained model that, though far from perfect, can generate actual English words, we can look at the predictions to continue learning more about how a text generation model works.

We can also take this as an opportunity to unpack the `def on_epoch_end` function to understand better how it works. 

In [75]:
# this is our joined clean sonnet data
text[:100]

'from fairest creatures we desire increase that thereby beautys rose might never die but as the riper'

In [76]:
# randomly pick a starting index 
# will be used to take a random sequence of chars from `text`
# run this cell a few times and you'll see `start_index` is random
start_index = random.randint(0, len(text) - dctk.maxlen - 1)
start_index

30895

In [77]:
# next use the randomly selected starting index to sample a sequence from the `text`

# this is our seed string (i.e., input sequence into the model)
generated = ''

# start the sentence at index `start_index` and include the next` dctk.maxlen` number of chars
sentence = text[start_index: start_index + dctk.maxlen]

# add to generated
generated += sentence

In [78]:
# display the "seed string" i.e. the input sequence into the model
print('----- Input seed: "' + sentence + '"')

----- Input seed: "end since every one "


In [79]:
# use model to predict what the next maxlen chars should be that follow the seed string
for i in range(maxlen):

    # shape of a single sample in a rank 3 tensor 
    x_dims = (1, dctk.maxlen, dctk.n_features)
    # create an array of zeros with shape x_dims
    # recall that python considers zeros and boolean FALSE as the same
    x_pred = np.zeros(x_dims)

    # create a seq vector for our randomly select sequence 
    # i.e. create a numerical encoding for each char in the sequence 
    for t, char in enumerate(sentence):
        # for sample 0 in seq index t and character `char` encode a 1 (which is the same as a TRUE)
        x_pred[0, t, dctk.char_int[char]] = 1

    # next, take the seq vector and pass into model to get a prediction of what the next char should be 
    preds = model.predict(x_pred, verbose=0)[0]
    # use the sample helper function to get index for next char 
    next_index = sample(preds)
    # use look up dict to get next char 
    next_char = dctk.int_char[next_index]

    # append next char to sequence 
    sentence = sentence[1:] + next_char 

In [80]:
# this is the seed string
generated

'end since every one '

In [81]:
# these are the maxlen chars that the model thinks should come after the seed stirng
sentence

'hath heart herar i c'

In [82]:
# how put it all together
generated + sentence

'end since every one hath heart herar i c'

# Resources and Stretch Goals

## Stretch Goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g., plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://www.tensorflow.org/text/tutorials/text_generation) - code for training an RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem and provides an example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN